In [26]:
#trendybox_sales.xlsx
import warnings
warnings.filterwarnings('ignore')
import pyodide_http
pyodide_http.patch_all()

import micropip
await micropip.install('pandas')
await micropip.install('openpyxl')
import pandas as pd

# Read the data from Excel file into dataframe
excel_file = pd.ExcelFile("trendybox_sales.xlsx", engine='openpyxl')

data = pd.read_excel(excel_file)

# Calculate 3 and 6 months savings
data['MA_3'] = data['Sales'].rolling(window=3).mean()
data['MA_6'] = data['Sales'].rolling(window=6).mean()

# View the forecasts
print(data.head(n=10))

# forecast using 3 and 6 month moving average
forecast_3 = data['MA_3'].dropna().iloc[-1]
forecast_6 = data['MA_6'].dropna().iloc[-1]

# Print the 3 and 6 month moving averages for January 2025
print(forecast_3)
print(forecast_6)

       Month  Sales        MA_3        MA_6
0 2023-01-31  492.0         NaN         NaN
1 2023-02-28  516.0         NaN         NaN
2 2023-03-31  474.0  494.000000         NaN
3 2023-04-30  506.0  498.666667         NaN
4 2023-05-31  524.0  501.333333         NaN
5 2023-06-30  477.0  502.333333  498.166667
6 2023-07-31  487.0  496.000000  497.333333
7 2023-08-31  496.0  486.666667  494.000000
8 2023-09-30  507.0  496.666667  499.500000
9 2023-10-31  486.0  496.333333  496.166667
472.0
491.8333333333333


In [27]:
# Remove any missing values from the data frame.
# prepare the data for the forecast accuracy measures calculations
# remove the missing values from the data frame so that MSE, MAE, and MAPE
# can be calculated.
data.dropna (inplace=True)
# Calculate and print the MSE for 3-month MA
mse_3 = ((data['Sales'] - data['MA_3'])** 2).mean()
print(f"The Mean Square Error (MSE) for 3-month MA is: {mse_3: .1f}")
# Calculate and print the MSE for 6-month MA
mse_6 = ((data['Sales'] - data['MA_6'])** 2).mean()
print(f"The Mean Square Error (MSE) for 6-month MA is: {mse_6: .1f}")

# Calculate and print the MAE for 3-month MA
mae_3 = (data['Sales'] - data['MA_3']).abs().mean()
print(f"The Mean Absolute Error (MAE) for 3-month MA is: {mae_3: .1f}")
# Calculate and print the MAE for 6-month MA
mae_6 = (data['Sales'] - data['MA_6']).abs().mean()
print(f"The Mean Absolute Error (MAE) for 6-month MA is: {mae_6: .1f}")

# Calculate and print the MAPE for 3-month MA
mape_3 = (abs(data['Sales'] - data['MA_3']) / data['Sales']).mean() * 100
print(f"The Mean Absolute Percentage Error (MAPE) for 3-month MA is: {mape_3: .1f}%")
# Calculate and print the MAPE for 6-month MA
mape_6 = (abs(data['Sales'] - data['MA_6']) / data['Sales']).mean() * 100
print(f"The Mean Absolute Percentage Error (MAPE) for 6-month MA is: {mape_6: .1f}%")

The Mean Square Error (MSE) for 3-month MA is:  305.3
The Mean Square Error (MSE) for 6-month MA is:  396.4
The Mean Absolute Error (MAE) for 3-month MA is:  14.4
The Mean Absolute Error (MAE) for 6-month MA is:  15.4
The Mean Absolute Percentage Error (MAPE) for 3-month MA is:  2.9%
The Mean Absolute Percentage Error (MAPE) for 6-month MA is:  3.2%


This indicates that the 3-month moving average is more responsive to recent changes in the sales data and provides more accurate forecasts for TrendyBox. Therefore, using the 3-month moving average for forecasting would likely result in better inventory management, financial planning, and overall decision-making for the company.

In [28]:
# Exponential Smoothing

# View the last observations in the data frame
data.tail()

# Importlibraries and modules
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
# Extract the Sales column
sales = data['Sales']
# Create and forecast values using the exponential smoothing model
# with a specified smoothing parameters, alpha es_model
es_model = SimpleExpSmoothing(sales).fit(smoothing_level=0.8)
# Print the forecasted/fitted values
print(es_model.fittedvalues)

/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/lib/python3.12/site-packages/pandas/util/_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)


5     477.000000
6     477.000000
7     485.000000
8     493.800000
9     504.360000
10    489.672000
11    495.534400
12    482.306880
13    501.261376
14    505.852275
15    490.770455
16    507.754091
17    477.550818
18    482.710164
19    486.142033
20    499.628407
21    535.925681
22    511.985136
23    473.597027
dtype: float64
